# Initial Value Problems

> # Let's begin!

## Question

### Code a program to tabulate the values of a differential equations using Euler's method.

> #### Notes for output
>
> Parameters:
>    - f (function): The differential equation dy/dx = f(x, y)
>    - x0 (float): Initial value of x
>    - y0 (float): Initial value of y
>    - h (float): Step size
>    - n (int): Number of steps
>
>    Returns:
>    - DataFrame containing the solution

In [ ]:
import numpy as np
from sympy import symbols, sympify, lambdify, Function, dsolve, Eq, solve
import pandas as pd
import matplotlib.pyplot as plt

def eulerMethod(f, x0, y0, h, n, exact_solution=None):
    x_values = np.zeros(n + 1)
    y_values = np.zeros(n + 1)

    x_values[0] = x0
    y_values[0] = y0

    for i in range(n):
        x_values[i + 1] = x_values[i] + h
        y_values[i + 1] = y_values[i] + h * f(x_values[i], y_values[i])

    intervals = [f"[{x_values[i]:.4f}, {x_values[i+1]:.4f}]" if i < n else "N/A" for i in range(n+1)]

    data = {
        "Iteration": list(range(n + 1)),
        "Interval": intervals,
        "x": x_values,
        "y": y_values
    }

    # Add error if exact solution is available
    if exact_solution is not None:
        errors = np.zeros(n + 1)
        for i in range(n + 1):
            try:
                exact_val = exact_solution(x_values[i])
                errors[i] = abs(y_values[i] - exact_val)
            except:
                errors[i] = np.nan
        data["Error Value"] = errors

    df = pd.DataFrame(data)
    return df

def exactSolution(eq_str, x0, y0):
    try:
        x = symbols('x')
        y = Function('y')(x)

        eq_expr = sympify(eq_str)
        diff_eq = Eq(y.diff(x), eq_expr.subs(symbols('y'), y))
        general_sol = dsolve(diff_eq, y)
        sol_expr = general_sol.rhs

        C1 = symbols('C1')
        eq_const = Eq(sol_expr.subs(x, x0), y0)
        const_values = solve(eq_const, C1)

        if const_values:
            particular_sol = sol_expr.subs(C1, const_values[0])
            exact_func = lambdify(x, particular_sol)
            return exact_func, str(particular_sol)
        else:
            return None, "Could not apply initial conditions"

    except Exception as e:
        return None, f"Could not derive exact solution: {e}"

def userInput():
    eq_str = input("Enter the differential equation (dy/dx) in terms of x and y: ")
    x0 = float(input("Enter initial x value (x0): "))
    y0 = float(input("Enter initial y value (y0): "))
    h = float(input("Enter step size (h): "))
    n = int(input("Enter number of steps (n): "))

    x, y = symbols('x y')
    expr = sympify(eq_str)
    f = lambdify((x, y), expr)
    exact_func, exact_str = exactSolution(eq_str, x0, y0)

    return f, x0, y0, h, n, exact_func, eq_str, exact_str

def solveIVP():
    f, x0, y0, h, n, exact_func, eq_str, exact_str = userInput()

    print((f"## Solving: dy/dx = {eq_str}"))
    print((f"Initial conditions: x0 = {x0}, y0 = {y0}"))
    print((f"Step size: h = {h}, Number of steps: n = {n}"))

    has_exact = False
    if exact_func:
        print((f"Exact solution: y = {exact_str}"))
        x_exact = np.linspace(x0, x0 + n*h, 100)
        try:
            y_exact = [exact_func(xi) for xi in x_exact]
            has_exact = True
        except:
            print(("Error calculating exact values"))
    else:
        print((f"Note: {exact_str}"))

    results = eulerMethod(f, x0, y0, h, n, exact_func)
    display(results)

    plt.figure(figsize=(10, 6))
    plt.plot(results['x'], results['y'], 'bo-', label='Euler Method')

    if has_exact:
        plt.plot(x_exact, y_exact, 'r-', label='Exact Solution')

    plt.grid(True)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(f"Solution to dy/dx = {eq_str}")
    plt.legend()
    plt.show()

solveIVP()